In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import predict
import base
import shop_classify as sc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [16]:
#读取测试集
X_train_df = pd.read_csv("./data/dataset/dataset/shop_classify/train/X_train.txt", sep='\t', index_col=0, encoding='UTF-8')
y_train_df = pd.read_csv("./data/dataset/dataset/shop_classify/train/y_train.txt", sep='\t', index_col=0, encoding='UTF-8')

In [6]:
%reload_ext autoreload

In [7]:
# kf = KFold(len(test_train_df), n_folds=5, shuffle=True, random_state=4)

In [8]:

#max_features=None, max_leaf_nodes=None

# for train_index, test_index in kf:
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X_train_df.loc[train_index], X_train_df.loc[test_index]
#     y_train, y_test = y_train_df.loc[train_index], y_train_df.loc[test_index]
    

In [17]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X_train_df.values, y_train_df['type'].values, random_state=0)              

In [18]:
X_train

array([[ -4,   0,   4, ..., -57,  -8,  -1],
       [ -1, -10,  11, ...,  -3, -16,   4],
       [-94,  13,  81, ..., -72,  -1,  76],
       ..., 
       [  6,   8, -15, ..., -13,  10, -35],
       [ -1,  22, -21, ..., -44,  52, -28],
       [  1, -26,  24, ...,   5, -41,  29]], dtype=int64)

In [19]:


# Set the parameters by cross-validation
tuned_parameters = {'learning_rate': [0.05, 0.07, 0.1], 'n_estimators': [80, 100, 120],
                     'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2], 'max_depth': [3]}


gbdt = GradientBoostingClassifier(random_state=0)

clf = GridSearchCV(gbdt, tuned_parameters, cv=4)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Best parameters set found on development set:

{'n_estimators': 100, 'min_samples_leaf': 1, 'learning_rate': 0.07, 'max_depth': 3, 'min_samples_split': 3}

Grid scores on development set:

0.793 (+/-0.028) for {'n_estimators': 80, 'min_samples_leaf': 1, 'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 2}
0.791 (+/-0.032) for {'n_estimators': 100, 'min_samples_leaf': 1, 'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 2}
0.793 (+/-0.029) for {'n_estimators': 120, 'min_samples_leaf': 1, 'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 2}
0.793 (+/-0.026) for {'n_estimators': 80, 'min_samples_leaf': 1, 'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 3}
0.793 (+/-0.029) for {'n_estimators': 100, 'min_samples_leaf': 1, 'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 3}
0.792 (+/-0.028) for {'n_estimators': 120, 'min_samples_leaf': 1, 'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 3}
0.793 (+/-0.025) for {'n_estimators': 8

In [20]:
X_test_df = pd.read_csv("./data/dataset/dataset/shop_classify/test/X_test.txt", sep='\t', index_col=0, encoding='UTF-8')

In [21]:
y_test = clf.predict(X_test_df.values)

In [22]:
y_test_df = pd.DataFrame(y_test)
y_test_df = y_test_df.rename(columns={0:'type'})
y_test_df.index.name = 'id'

In [23]:
y_test_df.to_csv("./data/dataset/dataset/shop_classify/test/y_test.txt", sep='\t', index=True, encoding='UTF-8')